<a href="https://colab.research.google.com/github/myrondza/Data-Science-Machine-Learning-Deep-Learning-AI-Guide-Algorithms/blob/master/ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ChatBot (Keras & Tensorflow)

In [0]:
import pickle
import numpy as np

In [267]:
a = [
    (['Myron','went','to','work'],['Is','Myron','at','work','?'],'yes'),
    (['Myron','did','not','go','to','office'],['Is','Myron','at','office','?'],'no'),
    (['John','did','not','go','to','the','party'],['Is','John','at','the','party','?'],'no'),
    (['Jack','went','to','the','party'],['Is','Jack','at','the','party','?'],'yes')
    ]

file_Name = "train_basic.txt"
fileObject = open(file_Name,'wb') 
pickle.dump(a,fileObject)  

fileObject.close()
fileObject = open(file_Name,'rb')  
train_data = pickle.load(fileObject)  
train_data

[(['Myron', 'went', 'to', 'work'], ['Is', 'Myron', 'at', 'work', '?'], 'yes'),
 (['Myron', 'did', 'not', 'go', 'to', 'office'],
  ['Is', 'Myron', 'at', 'office', '?'],
  'no'),
 (['John', 'did', 'not', 'go', 'to', 'the', 'party'],
  ['Is', 'John', 'at', 'the', 'party', '?'],
  'no'),
 (['Jack', 'went', 'to', 'the', 'party'],
  ['Is', 'Jack', 'at', 'the', 'party', '?'],
  'yes')]

In [268]:
len(train_data)

4

In [269]:
b = [
    (['Myron','did','not','go','to','office'],['Is','Myron','at','office','?'],'no')
    ]

file_Name = "test_basic.txt"
fileObject = open(file_Name,'wb') 
pickle.dump(b,fileObject)  

fileObject.close()
fileObject = open(file_Name,'rb')  
test_data = pickle.load(fileObject)  
test_data

[(['Myron', 'did', 'not', 'go', 'to', 'office'],
  ['Is', 'Myron', 'at', 'office', '?'],
  'no')]

In [270]:
len(test_data)

1

In [271]:
train_data[0]

(['Myron', 'went', 'to', 'work'], ['Is', 'Myron', 'at', 'work', '?'], 'yes')

In [272]:
test_data[0]

(['Myron', 'did', 'not', 'go', 'to', 'office'],
 ['Is', 'Myron', 'at', 'office', '?'],
 'no')

In [0]:
vocab = set()
for story, question, answer in train_data:
    vocab = vocab.union(set(story))
    vocab = vocab.union(set(question))

In [0]:
vocab.add('no')
vocab.add('yes')

In [275]:
vocab

{'?',
 'Is',
 'Jack',
 'John',
 'Myron',
 'at',
 'did',
 'go',
 'no',
 'not',
 'office',
 'party',
 'the',
 'to',
 'went',
 'work',
 'yes'}

In [0]:
vocab_len = len(vocab) + 1

In [0]:
all_data = test_data + train_data
all_story_lens = [len(data[0]) for data in all_data]
max_story_len = (max(all_story_lens))
max_question_len = max([len(data[1]) for data in all_data])

In [0]:
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

In [0]:
tokenizer = Tokenizer(filters = [])
tokenizer.fit_on_texts(vocab)

In [280]:
tokenizer.word_index

{'?': 9,
 'at': 14,
 'did': 10,
 'go': 16,
 'is': 7,
 'jack': 1,
 'john': 11,
 'myron': 6,
 'no': 5,
 'not': 3,
 'office': 2,
 'party': 8,
 'the': 15,
 'to': 13,
 'went': 12,
 'work': 17,
 'yes': 4}

In [0]:
train_story_text = []
train_question_text = []
train_answers = []

In [0]:
for story,question,answer in train_data:
    train_story_text.append(story)
    train_question_text.append(question) 
    train_answers.append(answer)

In [0]:
train_story_seq = tokenizer.texts_to_sequences(train_story_text)

In [0]:
def vectorize_stories(data,word_index = tokenizer.word_index, max_story_len = max_story_len, max_question_len = max_question_len):
    #vectorized stories:
    X = []
    #vectorized questions:
    Xq = []
    #vectorized answers:
    Y = []
    
    for story, question, answer in data:
        #Getting indexes for each word in the story
        x = [word_index[word.lower()] for word in story]
        #Getting indexes for each word in the story
        xq = [word_index[word.lower()] for word in question]
        #For the answers
        y = np.zeros(len(word_index) + 1) #Index 0 Reserved when padding the sequences
        y[word_index[answer]] = 1
        
        X.append(x)
        Xq.append(xq)
        Y.append(y)
        
    #Now we have to pad these sequences:
    return(pad_sequences(X,maxlen=max_story_len), pad_sequences(Xq, maxlen=max_question_len), np.array(Y))
       

In [0]:
inputs_train, questions_train, answers_train = vectorize_stories(train_data)

In [0]:
inputs_test, questions_test, answers_test = vectorize_stories(test_data)

In [287]:
inputs_train[0]

array([ 0,  0,  0,  6, 12, 13, 17], dtype=int32)

In [288]:
train_story_text[0]

['Myron', 'went', 'to', 'work']

In [289]:
train_story_seq[0]

[6, 12, 13, 17]

In [0]:
from keras.models import Sequential, Model
from keras.layers.embeddings import Embedding
from keras.layers import Input, Activation, Dense, Permute, Dropout, add, dot, concatenate, LSTM

In [0]:
input_sequence = Input((max_story_len,))
question = Input((max_question_len,))

In [0]:
input_encoder_m = Sequential()
input_encoder_m.add(Embedding(input_dim=vocab_len,output_dim = 64)) #From paper
input_encoder_m.add(Dropout(0.3))

input_encoder_c = Sequential()
input_encoder_c.add(Embedding(input_dim=vocab_len,output_dim = max_question_len)) #From paper
input_encoder_c.add(Dropout(0.3))

question_encoder = Sequential()
question_encoder.add(Embedding(input_dim=vocab_len,output_dim = 64,input_length=max_question_len)) #From paper
question_encoder.add(Dropout(0.3))

input_encoded_m = input_encoder_m(input_sequence)
input_encoded_c = input_encoder_c(input_sequence)
question_encoded = question_encoder(question)

match = dot([input_encoded_m,question_encoded], axes = (2,2))
match = Activation('softmax')(match)

response = add([match,input_encoded_c])
response = Permute((2,1))(response)

answer = concatenate([response, question_encoded])

In [293]:
answer

<tf.Tensor 'concatenate_7/concat:0' shape=(?, 6, 71) dtype=float32>

In [294]:
answer = LSTM(32)(answer)
answer = Dropout(0.5)(answer)
answer = Dense(vocab_len)(answer)
answer = Activation('softmax')(answer)
model = Model([input_sequence,question], answer)
model.compile(optimizer='rmsprop', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_15 (InputLayer)           (None, 7)            0                                            
__________________________________________________________________________________________________
input_16 (InputLayer)           (None, 6)            0                                            
__________________________________________________________________________________________________
sequential_19 (Sequential)      multiple             1152        input_15[0][0]                   
__________________________________________________________________________________________________
sequential_21 (Sequential)      (None, 6, 64)        1152        input_16[0][0]                   
__________________________________________________________________________________________________
dot_7 (Dot

In [295]:
history = model.fit([inputs_train,questions_train],answers_train, batch_size = 32, epochs = 1000, validation_data = ([inputs_test,questions_test],answers_test))

Train on 4 samples, validate on 1 samples
Epoch 1/1000
4/4 [==============================] - 2s 582ms/step - loss: 2.8691 - acc: 0.0000e+00 - val_loss: 2.8195 - val_acc: 0.0000e+00
Epoch 2/1000
4/4 [==============================] - 0s 3ms/step - loss: 2.8265 - acc: 0.2500 - val_loss: 2.7781 - val_acc: 1.0000
Epoch 3/1000
4/4 [==============================] - 0s 3ms/step - loss: 2.8019 - acc: 0.5000 - val_loss: 2.7507 - val_acc: 1.0000
Epoch 4/1000
4/4 [==============================] - 0s 3ms/step - loss: 2.8046 - acc: 0.0000e+00 - val_loss: 2.7175 - val_acc: 1.0000
Epoch 5/1000
4/4 [==============================] - 0s 3ms/step - loss: 2.7285 - acc: 0.2500 - val_loss: 2.6836 - val_acc: 1.0000
Epoch 6/1000
4/4 [==============================] - 0s 3ms/step - loss: 2.7142 - acc: 0.5000 - val_loss: 2.6440 - val_acc: 1.0000
Epoch 7/1000
4/4 [==============================] - 0s 3ms/step - loss: 2.7257 - acc: 0.5000 - val_loss: 2.6117 - val_acc: 1.0000
Epoch 8/1000
4/4 [================

In [0]:
filename = 'chatbot_epochs.h5'
model.save(filename)

In [0]:
model.load_weights('chatbot_epochs.h5')

In [0]:
pred_results = model.predict(([inputs_test,questions_test]))

In [299]:
test_data[0]

(['Myron', 'did', 'not', 'go', 'to', 'office'],
 ['Is', 'Myron', 'at', 'office', '?'],
 'no')

In [300]:
pred_results[0]

array([3.9488292e-08, 2.9919708e-08, 3.0460281e-08, 1.3006689e-08,
       8.3579693e-07, 9.9999857e-01, 5.3202410e-08, 4.1330836e-08,
       5.2020752e-08, 4.5391399e-08, 2.2997231e-08, 3.5505060e-08,
       2.0221108e-08, 1.5313587e-08, 2.3289006e-08, 2.8422717e-08,
       3.0997668e-08, 4.3086185e-08], dtype=float32)

In [301]:
val_max = np.argmax(pred_results[0])
for key,val in tokenizer.word_index.items():
    if val == val_max:
        k = key
print(k)

no


In [302]:
pred_results[0][val_max]

0.99999857

In [305]:
my_story = 'Myron went to the party'
my_story.split()

['Myron', 'went', 'to', 'the', 'party']

In [306]:
my_question = 'Is Myron at the party ?'
my_question.split()

['Is', 'Myron', 'at', 'the', 'party', '?']

In [0]:
my_data = [(my_story.split(), my_question.split(),'yes')]

In [0]:
my_story, my_ques, my_ans = vectorize_stories(my_data)

In [0]:
pred_results = model.predict(([my_story,my_ques]))

In [0]:
val_max = np.argmax(pred_results[0])

In [311]:
for key,val in tokenizer.word_index.items():
    if val == val_max:
        k = key
print(k)

yes


In [312]:
pred_results[0][val_max]

0.99999845